### User Define Model Class
- perents class: nn.Module
- 필수 오버라이딩
    - __init__(): 모델 층 구성, 전개
    - forward(): 순방향 학습 진행 코드
 - 동적 모델
    - container 모듈 둥 nn.ModuleList() 사용해서 동적으로 Layer 추가
        - 연결이 안되어있다??
        - Sequential은 연결되어있지만 저 친구는 아닌ㅁ

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optima

from torchinfo import summary
from torchmetrics.regression import R2Score, MeanSquaredError
from torchmetrics.classification import F1Score, Accuracy, ConfusionMatrix

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
torch.manual_seed(14)

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

- basic create ANN model
    - input layer: input node is len(feature)
    - output layer: output node is len(label)
    - hidden layer: fixed

- model structure
    - datasets: feature dynamic, label dynamic, regression
    - input layer   : input dynamic,   output dynamic,  activation function: ReLU, sigmoid (To solve gradient vanishing problem)
    - hidden layer  : dynamic, activation function: ReLu, sigmoid
    - output layer  : input dynamic, output dynamic,   activation function: None (마지막에는 확률값을 기반으로 결과를 도출하기에, 분류는 필요함. sigmoid, softmax)
    - input, output, hidden 모든것이 동적, hidden perceptron은 동일

- - -
- 모델 이름: DynamicModel
- 부모 class: nn.Module
- parameter: in_in, out_out, h_inout, h_cnt
- - -

In [44]:
# hidden layer가 동적인 모델
class DynamicMyModel(nn.Module):
    # callback function
    def __init__(self, in_in, out_out, h_inout, h_cnt):
        super().__init__()
        
        self.input_layer = nn.Linear(in_in, h_inout)
        self.h_layers = nn.ModuleList([nn.Linear(h_inout, h_inout) for _ in range(h_cnt)])
        # nn.ModuleList: 층을 동적으로 넣을때
        self.output_layer = nn.Linear(h_inout, out_out)
        
    # forward learning
    # callback funtion
    # 전달인자: 학습용 데이터셋
    def forward(self, x):
        y = self.input_layer(x)
        y = F.relu(y)
        for layer in self.h_layers:
            y = layer(y)
            y = F.relu(y)
        y = self.output_layer(y)
        return y

In [45]:
model = DynamicMyModel(3, 2, 5, 10)


In [46]:
for name, param in model.named_parameters():
    print(f"[{name}]\n{param}\n")

[input_layer.weight]
Parameter containing:
tensor([[-0.5249,  0.0662, -0.1112],
        [ 0.1251, -0.4170,  0.0041],
        [-0.5362, -0.5040, -0.3700],
        [ 0.5637,  0.3859, -0.3439],
        [ 0.5543, -0.1822,  0.2838]], requires_grad=True)

[input_layer.bias]
Parameter containing:
tensor([ 0.5349,  0.2100, -0.2777, -0.5025, -0.2382], requires_grad=True)

[h_layers.0.weight]
Parameter containing:
tensor([[ 0.0763, -0.0913,  0.4168, -0.0932,  0.1215],
        [ 0.0350, -0.3118,  0.1556,  0.4144, -0.2095],
        [ 0.1242,  0.3850, -0.0121, -0.4359,  0.3033],
        [ 0.1386, -0.2237,  0.2826, -0.0336, -0.2064],
        [ 0.3760, -0.1629, -0.2974,  0.2142, -0.1268]], requires_grad=True)

[h_layers.0.bias]
Parameter containing:
tensor([-0.0146, -0.1465,  0.0500, -0.1874, -0.2950], requires_grad=True)

[h_layers.1.weight]
Parameter containing:
tensor([[ 0.3180, -0.0118,  0.1843,  0.1905,  0.3439],
        [ 0.0266, -0.2354, -0.1597, -0.1930, -0.3910],
        [-0.4094,  0.0720,  

In [49]:
data_ts = torch.FloatTensor([[1, 3, 5], [2, 4, 6], [3, 5, 7], [4, 6, 8]])
target_ts = torch.FloatTensor([[7, 9], [8, 10], [9, 11], [10, 18]])

In [50]:
y_pred = model(data_ts)
print(y_pred)

tensor([[ 0.0747, -0.4718],
        [ 0.0747, -0.4718],
        [ 0.0747, -0.4718],
        [ 0.0747, -0.4718]], grad_fn=<AddmmBackward0>)
